In [3]:
%load_ext autoreload
%autoreload 2
from invest_data import load_data, stickers

data = load_data()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
SPY = data['SPY']
SPY

,Date,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,,
1996-12-31,1996-12-31,75.375000,73.843750,75.281250,73.843750,1378100.0,48.133801
1997-01-02,1997-01-02,74.375000,72.750000,74.375000,74.031250,2031900.0,48.256046
1997-01-03,1997-01-03,75.125000,74.078125,74.375000,75.093750,2123200.0,48.948616
1997-01-06,1997-01-06,75.437500,74.312500,75.093750,74.437500,1374100.0,48.520851
1997-01-07,1997-01-07,75.468750,74.125000,74.437500,75.343750,939000.0,49.111538
...,...,...,...,...,...,...,...
2020-10-26,2020-10-26,342.980011,335.619995,342.130005,339.390015,91473000.0,339.390015
2020-10-27,2020-10-27,340.119995,337.989990,339.760010,338.220001,65994100.0,338.220001
2020-10-28,2020-10-28,338.250000,326.130005,332.100006,326.660004,127094300.0,326.660004


In [24]:
from datetime import datetime, timedelta
import math
from common import get_price

def strategy_monthly(volum, SPY_sum, months_to_breakeven, high_dt, high):
    for i in range(months_to_breakeven):
        dt = high_dt + timedelta(days=(30.4*(i+1)))
        dt_str = dt.strftime('%Y-%m-%d')
        
        while not dt_str in SPY.index:
            dt = dt + timedelta(days=1)
            dt_str = dt.strftime('%Y-%m-%d')
        SPY_sum += SPY.loc[dt_str]['Adj Close']
        volum += 1
    
    return (SPY_sum, volum)

def strategy_percentage(volum, SPY_sum, months_to_breakeven, high_dt, high):
    tmp = high
    weight = 2.5
    for index, row in SPY.iterrows():
        if row['Adj Close'] < tmp * 0.9:
            tmp = tmp * 0.9
            SPY_sum += row['Adj Close'] * weight
            volum += weight
            weight += 0.5
        if row['Adj Close'] > tmp * 1.1:
            tmp = tmp * 1.1
            SPY_sum += row['Adj Close'] * weight
            volum += weight
            weight -= 0.5
        if tmp > high:
            break
        
    return (SPY_sum, volum)

def strategy_monthly_dynamic(volum, SPY_sum, months_to_breakeven, high_dt, high):
    for i in range(months_to_breakeven):
        dt = high_dt + timedelta(days=(30.4*(i+1)))
        dt_str = dt.strftime('%Y-%m-%d')
        val = get_price(SPY, dt_str)
        diff_percent = (high - val)/high
        weight = 0.5
        weight = weight*(1+diff_percent*4)

        SPY_sum += val*weight
        volum += weight
    
    return (SPY_sum, volum)

def get_v(ratio):
    v = 1
    if ratio < 1:
        v += 0.6 / (ratio**2)
    else:
        v -= 0.3 * (ratio**2)
    return v

def strategy_monthly_dynamic1(volum, SPY_sum, months_to_breakeven, high_dt, high):
    for i in range(months_to_breakeven):
        dt = high_dt + timedelta(days=(30.4*(i+1)))
        dt_str = dt.strftime('%Y-%m-%d')
        val = get_price(SPY, dt_str)
        
        dt_1m_before = dt - timedelta(days=30.4*6)
        ratio = val/get_price(SPY, dt_1m_before.strftime('%Y-%m-%d'))
        weight = get_v(ratio)
        
        dt_1m_before = dt - timedelta(days=30.4*3)
        ratio = val/get_price(SPY, dt_1m_before.strftime('%Y-%m-%d'))
        ratio = math.sqrt(ratio)
        weight /= ratio

        dt_1m_before = dt - timedelta(days=30.4)
        ratio = val/get_price(SPY, dt_1m_before.strftime('%Y-%m-%d'))
        ratio = math.sqrt(ratio)
        ratio = math.sqrt(ratio)
        weight /= ratio

        SPY_sum += val*weight
        volum += weight
    
    return (SPY_sum, volum)

In [25]:
def find_adjust_breakeven(start_dt, low_dt, s, v):
    avg_price = s/v
    print('sum:' + str(s) + ', volum:' + str(v) + ', avg price:' + str(avg_price))
    for index, row in SPY.iterrows():
        if index <= low_dt:
            continue
        if row['Adj Close'] > avg_price:
            months_to_adj_breakeven = index - start_dt
            months_to_adj_breakeven = math.ceil(months_to_adj_breakeven.days/30.4)
            return months_to_adj_breakeven

def analyze_crisis(msg, start_time):
    print('========== analyzing ' + msg + ' ' + start_time + ' ========== ')
    
    high = SPY.loc[start_time]['Adj Close'] #2008 最高点开始套牢
    print('start: ' + str(high))
    start_dt = datetime.strptime(start_time, '%Y-%m-%d')
    
    low_dt = start_dt
    low = high
    breakeven_dt = start_dt
    flag_crisis_start = False
    for index, row in SPY.iterrows():
        if index <= start_dt:
            continue
        flag_crisis_start = True
        val = row['Adj Close']
        if val < low:
            low = val
            low_dt = index
        if val > high and flag_crisis_start:
            print(val)
            breakeven_dt = index
            break
    
    print('low: ' + str(low))
    loss = (low - high)/high
    print('loss: ' + str(loss))
    
    months_to_low = low_dt - start_dt
    months_to_low = math.ceil(months_to_low.days/30.4)
    print("months to low: " + str(months_to_low))

    months_to_breakeven = breakeven_dt - start_dt
    months_to_breakeven = math.ceil(months_to_breakeven.days/30.4)
    print("months to breakeven: " + str(months_to_breakeven)+'\r\n')
    
    # assuming initial invest is equal to 5 months
    volum = 5
    SPY_sum = volum * high
    
    print('strategy monthly => ')
    s, v = strategy_monthly(volum, SPY_sum, months_to_breakeven, start_dt, high)
    months_to_adj_breakeven = find_adjust_breakeven(start_dt, low_dt, s, v)
    print("months to adj breakeven: " + str(months_to_adj_breakeven)+'\r\n')
    
    print('strategy percentage => ')
    s2, v2 = strategy_percentage(volum, SPY_sum, months_to_breakeven, start_dt, high)
    months_to_adj_breakeven2 = find_adjust_breakeven(start_dt, low_dt, s2, v2)
    print("months to adj breakeven: " + str(months_to_adj_breakeven2)+'\r\n')
    
    print('strategy percentage dynamic => ')
    s3, v3 = strategy_monthly_dynamic(volum, SPY_sum, months_to_breakeven, start_dt, high)
    months_to_adj_breakeven3 = find_adjust_breakeven(start_dt, low_dt, s3, v3)
    print("months to adj breakeven: " + str(months_to_adj_breakeven3)+'\r\n')
    
    print('strategy percentage dynamic1 => ')
    s4, v4 = strategy_monthly_dynamic1(volum, SPY_sum, months_to_breakeven, start_dt, high)
    months_to_adj_breakeven4 = find_adjust_breakeven(start_dt, low_dt, s4, v4)
    print("months to adj breakeven: " + str(months_to_adj_breakeven4)+'\r\n')
    
analyze_crisis('2008 crisis', start_time='2007-10-12')

========== analyzing 2008 crisis 2007-10-12 ========== 
start: 119.794189453125
120.45427703857422
low: 53.73193740844727
loss: -0.5514645772575442
months to low: 17
months to breakeven: 59

strategy monthly => 
sum:6273.850410461426, volum:64, avg price:98.02891266345978
months to adj breakeven: 31

strategy percentage => 
sum:16620.99846458435, volum:225.5, avg price:73.70731026423215
months to adj breakeven: 19

strategy percentage dynamic => 
sum:5443.513707202145, volum:57.75618163781878, avg price:94.24988897877088
months to adj breakeven: 30

strategy percentage dynamic1 => 
sum:7241.221844491962, volum:77.02690945813617, avg price:94.00898848768608
months to adj breakeven: 30



In [26]:
analyze_crisis('2000 crisis', start_time='2000-09-1')

========== analyzing 2000 crisis 2000-09-1 ========== 
start: 103.85742950439452
104.06795501708984
low: 54.75950622558594
loss: -0.47274348607608374
months to low: 26
months to breakeven: 74

strategy monthly => 
sum:6556.4902420043945, volum:79, avg price:82.99354736714423
months to adj breakeven: 42

strategy percentage => 
sum:8879.26626777649, volum:129.0, avg price:68.8315214556317
months to adj breakeven: 34

strategy percentage dynamic => 
sum:5947.645239793567, volum:73.74056390011134, avg price:80.65635689808695
months to adj breakeven: 41

strategy percentage dynamic1 => 
sum:7144.477203316912, volum:88.39581503805, avg price:80.82370415660029
months to adj breakeven: 41

